# Search for Self-Similar Solutions

In [ ]:
import sys
sys.path.append('../src')
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import newton
from sklearn.neighbors import KernelDensity
from scipy.io import savemat
from Davidenko import Davidenko

In [ ]:
% matplotlib inline

Start with the heat equation.

Define the initial conditions and objective function.

In [ ]:
minpos, maxpos, points, its = 0.9, 1.05, 200, 119
def LennardJones(x):
    return (1 - 2 * x ** 6) / x ** 12
def dLennardJones(x):
    return 12 * (x ** 6 -1) / x ** 13
def ddLennardJones(x):
    return (156 - 84 * x ** 6) / x ** 14

Define a Newton Solver method to compute the new positions after each iteration.

In [ ]:
def NewtonSimilarSolver(x, df, ddf, its):
    positions = np.empty((its + 1, x.shape[0]))
    positions[0, :] = x
    for i in range(its):
        for j in range(positions.shape[1]):
            positions[i+1, j] = newton(df, positions[i, j], ddf, tol=float('Infinity'), maxiter=1)
        positions[i+1, :] -= min(positions[i+1, :])
        positions[i+1, :] *= (max(positions[i, :] - min(positions[i, :]))) / max(positions[i+1, :])
        positions[i+1, :] += min(positions[i, :])
    return positions

In [ ]:
positions = NewtonSimilarSolver(np.linspace(minpos, maxpos, points), dLennardJones, ddLennardJones, its)

Perform KDE on the positions.

In [ ]:
kde = KernelDensity(bandwidth=3 * (maxpos - minpos)/points, kernel='gaussian')
samples = np.linspace(minpos, maxpos, points)
density = np.empty((its + 1, points))
for i in range(its + 1):
    kde.fit(positions[i, :][:, np.newaxis])
    density[i, :] = np.exp(kde.score_samples(samples[:, np.newaxis]))

In [ ]:
#fig, ax = plt.subplots(its + 1, 1, figsize=(4, 8))
#for i in range(its + 1):
#    ax[i].plot(samples, density[i, :])
#    if i != its:
#        ax[i].set_xticks([])
#plt.tight_layout()

In [ ]:
savemat('Self-Similar-Newton', {'Density' : density, 'Positions' : samples})